모델 버전2를 써서 

In [143]:
from ultralytics import YOLO
model = YOLO("modelFolder/model_v2.pt")
#사진해상도 1280*720
results = model(source="w640h360/8_Color.png", device="cpu",project="results",name="output",save=True)


image 1/1 c:\Team_Project-1\w640h360\8_Color.png: 384x640 1 bongji, 3 boxs, 1 energdrink, 355.5ms
Speed: 2.0ms preprocess, 355.5ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)
Results saved to results\output7


In [144]:
for result in results:
    xy_list = result.masks.xyn  

In [151]:
import numpy as np
import open3d as o3d
pcd = o3d.io.read_point_cloud('w640h360/8.ply')

aabb = pcd.get_axis_aligned_bounding_box()
aabb.color = (1, 0, 0)
obb = pcd.get_oriented_bounding_box()
obb.color = (0, 1, 0)

# visualize
o3d.visualization.draw_geometries([pcd, aabb, obb])
# obb.color(1, 0, 0)
print(aabb.get_max_bound())
print(aabb.get_min_bound())
print(aabb)
# o3d.visualization.draw_geometries([pcd,obb])

[    0.69287     0.37817    -0.43164]
[   -0.64551    -0.40405    -0.70459]
AxisAlignedBoundingBox: min: (-0.645508, -0.404053, -0.70459), max: (0.692871, 0.378174, -0.431641)


In [146]:
import numpy as np
import math

def yaw_pitch_roll_to_rotation_matrix(yaw, pitch, roll):
    # 각도를 라디안으로 변환
    yaw_rad = math.radians(yaw)
    pitch_rad = math.radians(pitch)
    roll_rad = math.radians(roll)

    # Yaw, Pitch, Roll 회전 행렬 계산
    yaw_matrix = np.array([
        [math.cos(yaw_rad), -math.sin(yaw_rad), 0],
        [math.sin(yaw_rad), math.cos(yaw_rad), 0],
        [0, 0, 1]
    ])

    pitch_matrix = np.array([
        [math.cos(pitch_rad), 0, math.sin(pitch_rad)],
        [0, 1, 0],
        [-math.sin(pitch_rad), 0, math.cos(pitch_rad)]
    ])

    roll_matrix = np.array([
        [1, 0, 0],
        [0, math.cos(roll_rad), -math.sin(roll_rad)],
        [0, math.sin(roll_rad), math.cos(roll_rad)]
    ])

    # 회전 행렬을 순서대로 곱해서 최종 회전 행렬 생성
    rotation_matrix = np.dot(np.dot(roll_matrix, pitch_matrix), yaw_matrix)
    return rotation_matrix

# 예시 각도 (단위: 도)
# yaw_angle = 30
# pitch_angle = 45
# roll_angle = 60
yaw_angle = 180
pitch_angle = 0
roll_angle = 0

# 회전 행렬 생성
rotation_matrix = yaw_pitch_roll_to_rotation_matrix(yaw_angle, pitch_angle, roll_angle)

print(rotation_matrix)

[[         -1 -1.2246e-16           0]
 [ 1.2246e-16          -1           0]
 [          0           0           1]]


In [147]:
import numpy as np
import open3d as o3d
import math

# # read ply file
# pcd = o3d.io.read_point_cloud('w640h360/2.ply')

# 
# rotate = np.identity(3)  
# size = np.ones(3) * 1.35 # trying to create a bounding box below 1 unit
# center = np.zeros(3) 

# 회전 행렬 참고자료 : https://en.wikipedia.org/wiki/Rotation_matrix
yaw_angle = 0  # + : crop 영역 반시계 방향 회전(카메라 시점), - : crop 영역 시계 방향 회전(카메라 시점)
pitch_angle = 0
roll_angle = 0

rotation_matrix = yaw_pitch_roll_to_rotation_matrix(yaw_angle, pitch_angle, roll_angle)
size = [0.95,  # 너비(카메라 기준, 2d width) 1.28 보다 작아지면 우측 잘리기 시작함
        0.45,  # 높이(카메라 기준, 2d height) 0.849 보다 작아지면 아래쪽 잘리기 시작함, 0.00875 = 2 pixel 거리
        1.500]  # 깊이(카메라 기준 거리, 3d depth) 1.387 보다 작아지면 좌측 아래쪽 바닥 잘리기 시작함
                # 바닥 제거 1.345

center = [-0.015,  # + : crop 영역이 우측으로 이동, - : crop 영역이 좌측으로 이동 (-0.64551 ~ 0.69287), 합 1.33838
          0.0,  # + : crop 영역이 위쪽으로 이동, - : crop 영역이 아래쪽으로 이동 ( -0.40405 ~ 0.37817)  합 0.78222
          0.0,] # + : crop 영역이 카메라쪽으로 이동, - : crop 영역이 아래쪽으로 이동
obb = o3d.geometry.OrientedBoundingBox(center, rotation_matrix, size) # or you can use axis aligned bounding box class

print(rotation_matrix)
print(size)
print(center)

crop_pcd0 = pcd.crop(obb)
# print(pcd)
# visualize
o3d.visualization.draw_geometries([crop_pcd0])
# o3d.io.write_point_cloud("cropply/croppedtest2.ply", crop_pcd)
# pcd = o3d.io.read_point_cloud('your_ply_file.ply')




[[          1           0           0]
 [          0           1           0]
 [          0           0           1]]
[0.95, 0.45, 1.5]
[-0.015, 0.0, 0.0]


In [148]:
import numpy as np
import open3d as o3d

# 원하는 부분을 크롭한 포인트 클라우드 데이터 (crop_pcd0로 가정)
# crop_pcd0 = ...

# 포인트 클라우드 좌표 추출
points = np.asarray(crop_pcd0.points)

# x, y, z 좌표를 0.0에서 1.0로 스케일 및 이동
x_min = np.min(points[:, 0])
y_min = np.min(points[:, 1])
# z_min = np.min(points[:, 2])
x_range = np.max(points[:, 0]) - x_min
y_range = np.max(points[:, 1]) - y_min
# z_range = np.max(points[:, 2]) - z_min

scaled_x = (points[:, 0] - x_min) / x_range
scaled_y = (points[:, 1] - y_min) / y_range
# scaled_z = (points[:, 2] - z_min) / z_range

# 변환된 좌표로 업데이트
points[:, 0] = scaled_x
points[:, 1] = scaled_y
# points[:, 2] = scaled_z

# 업데이트된 포인트 클라우드로 저장 또는 시각화
crop_pcd0.points = o3d.utility.Vector3dVector(points)

# 시각화 (선택 사항)
o3d.visualization.draw_geometries([crop_pcd0])


In [153]:
import numpy as np
import open3d as o3d

# xy_list에서 첫 번째 배열을 사용하여 새로운 배열 생성
xy = []
for i in range(len(xy_list)):
    new_xyz_list = np.c_[xy_list[i], np.full(xy_list[i].shape[0], 0.05)]
    xy.append(new_xyz_list)
    
# new_xyz_list = np.c_[xy_list[1], np.full(xy_list[1].shape[0], 0.05)]

for xyn in xy:
    corners = np.array(xyn)

#PLY파일이 원본 크기의 0.5배이기 때문에 모든 x,y,z좌표에 0.5 곱해서 비율 맞추기

corners[:, 1] = 1 - corners[:, 1]


# corners 배열의 데이터 유형을 float64로 변환
bounding_polygon = corners.astype("float64")

# SelectionPolygonVolume 생성
vol = o3d.visualization.SelectionPolygonVolume()

# 다각형을 어떤 축을 기준으로 정렬할 것인지 지정해야 합니다.
# 여기서는 "Z" 축을 선택합니다. 최대값은 다각형 꼭지점 중 최대 Z 값,
# 최소값은 다각형 꼭지점 중 최소 Z 값으로 설정합니다.
axis_num = 2
vol.orthogonal_axis = "Z"
vol.axis_max = 10.0
vol.axis_min = -10.0

# Z 값은 설정한 vol.axis_max 및 vol.axis_min 값으로 이미 지정되었으므로
# 모든 Z 값을 0으로 설정합니다.
bounding_polygon[:, axis_num] = 0

# np.array를 Vector3dVector로 변환합니다.
vol.bounding_polygon = o3d.utility.Vector3dVector(bounding_polygon)

print("vol = ", vol)
print("vol.orthogonal_axis = ", vol.orthogonal_axis)
print("vol.axis_min = ", vol.axis_min)
print("vol.axis_max = ", vol.axis_max)
print("vol.bounding_polygon = \n", np.asarray(vol.bounding_polygon))

# Vector3dVector를 사용하여 포인트 클라우드를 자릅니다.
crop_pcd = vol.crop_point_cloud(crop_pcd0)

# 정육면체의 선 정보 (꼭지점을 연결하는 라인 인덱스)
lines = list()
for i in range(len(corners)):
    lines.append([i, (i+1) % len(corners)])

print(lines)

# 라인 정보 생성
lineset = o3d.geometry.LineSet()
lineset.points = o3d.utility.Vector3dVector(corners)
lineset.lines = o3d.utility.Vector2iVector(lines)

# 시각화
o3d.visualization.draw_geometries([crop_pcd0, lineset])
o3d.visualization.draw_geometries([crop_pcd, lineset])




vol =  SelectionPolygonVolume, access its members:
orthogonal_axis, bounding_polygon, axis_min, axis_max
vol.orthogonal_axis =  Z
vol.axis_min =  -10.0
vol.axis_max =  10.0
vol.bounding_polygon = 
 [[    0.52656     0.84444           0]
 [      0.525     0.84167           0]
 [      0.525     0.83889           0]
 [    0.51875     0.82778           0]
 [    0.51875     0.81667           0]
 [     0.5125     0.80556           0]
 [     0.5125     0.79444           0]
 [    0.51094     0.79167           0]
 [    0.51094     0.78889           0]
 [    0.50937     0.78611           0]
 [    0.50937     0.78056           0]
 [    0.50781     0.77778           0]
 [    0.50781       0.775           0]
 [    0.50937     0.77222           0]
 [    0.51719     0.77222           0]
 [    0.52031     0.77778           0]
 [    0.52344     0.77778           0]
 [      0.525     0.78056           0]
 [    0.54219     0.78056           0]
 [    0.54375     0.77778           0]
 [    0.54531     0.77

In [152]:
import numpy as np
import open3d as o3d

# xy_list에서 첫 번째 배열을 사용하여 새로운 배열 생성
xy = []
for i in range(len(xy_list)):
    new_xyz_list = np.c_[xy_list[i], np.full(xy_list[i].shape[0], 0.05)]
    xy.append(new_xyz_list)

# 도형을 저장할 리스트를 생성합니다.
shapes = []

# 각 배열을 도형으로 만들어서 shapes 리스트에 추가합니다.
for xyn in xy:
    corners = np.array(xyn)
    
    # PLY파일이 원본 크기의 0.5배이기 때문에 모든 x,y,z좌표에 0.5 곱해서 비율 맞춤
    corners[:, 1] = 1 - corners[:, 1]
    
    # corners 배열의 데이터 유형을 float64로 변환
    bounding_polygon = corners.astype("float64")
    
    # SelectionPolygonVolume 생성
    vol = o3d.visualization.SelectionPolygonVolume()
    
    # 다각형을 어떤 축을 기준으로 정렬할 것인지 지정
    axis_num = 2
    vol.orthogonal_axis = "Z"
    vol.axis_max = 10.0
    vol.axis_min = -10.0
    
    # Z 값을 설정한 vol.axis_max 및 vol.axis_min 값으로 이미 지정되었으므로 모든 Z 값을 0으로 설정
    bounding_polygon[:, axis_num] = 0
    
    # np.array를 Vector3dVector로 변환
    vol.bounding_polygon = o3d.utility.Vector3dVector(bounding_polygon)
    
    # Vector3dVector를 사용하여 포인트 클라우드를 자름
    crop_pcd = vol.crop_point_cloud(crop_pcd0)
    
    # 도형을 shapes 리스트에 추가
    shapes.append((crop_pcd, vol))

# 모든 도형을 결합한 포인트 클라우드 생성
combined_pcd = o3d.geometry.PointCloud()
for shape_pcd, _ in shapes:
    combined_pcd += shape_pcd

# 시각화
lines = list()
for j in range(len(corners)):
    lines.append([j, (j+1) % len(corners)])
lineset = o3d.geometry.LineSet()
lineset.points = o3d.utility.Vector3dVector(corners)
lineset.lines = o3d.utility.Vector2iVector(lines)
o3d.visualization.draw_geometries([combined_pcd, lineset])
